[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//Insisted/CH2-PS020_FitSync/blob/main/ML/model/FitSync-Recommender.ipynb)

In [1]:
try:
    import zipfile
    from google.colab import drive

    drive.mount('/content/drive')

    token = '/content/drive/MyDrive/Colab Notebooks/kaggle.json'
    dataset_path = '/content/megaGymDataset.csv'

    !pip install kaggle --quiet

    !mkdir ~/.kaggle
    !cp "{token}" ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    !kaggle datasets download -d niharika41298/gym-exercise-data

    zip_ref = zipfile.ZipFile('gym-exercise-data.zip', 'r')
    zip_ref.extractall('/content')
    zip_ref.close()
except ImportError:
    dataset_path = '../data/megaGymDataset.csv'

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(dataset_path) # https://www.kaggle.com/datasets/niharika41298/gym-exercise-data

df.head()

,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918 entries, 0 to 2917
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2918 non-null   int64  
 1   Title       2918 non-null   object 
 2   Desc        1368 non-null   object 
 3   Type        2918 non-null   object 
 4   BodyPart    2918 non-null   object 
 5   Equipment   2886 non-null   object 
 6   Level       2918 non-null   object 
 7   Rating      1031 non-null   float64
 8   RatingDesc  862 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 205.3+ KB


In [5]:
df.isna().sum()

Unnamed: 0       0
Title            0
Desc          1550
Type             0
BodyPart         0
Equipment       32
Level            0
Rating        1887
RatingDesc    2056
dtype: int64

In [6]:
# Convert to categorical

df['Title'] = pd.Categorical(df.Title)
df['Type'] = pd.Categorical(df.Type)
df['BodyPart'] = pd.Categorical(df.BodyPart)
df['Level'] = pd.Categorical(df.Level)

In [7]:
df.Equipment.unique()

array(['Bands', 'Barbell', 'Kettlebells', 'Dumbbell', 'Other', 'Cable',
       'Machine', 'Body Only', 'Medicine Ball', nan, 'Exercise Ball',
       'Foam Roll', 'E-Z Curl Bar'], dtype=object)

In [8]:
df = df.loc[(df.Equipment == 'Body Only')] # (data.Level == 'Beginner'), (data.Desc.notna())

df.drop(['Unnamed: 0', 'Desc', 'Equipment', 'RatingDesc'], axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1078 entries, 185 to 2909
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Title     1078 non-null   category
 1   Type      1078 non-null   category
 2   BodyPart  1078 non-null   category
 3   Level     1078 non-null   category
 4   Rating    269 non-null    float64 
dtypes: category(4), float64(1)
memory usage: 110.5 KB


In [9]:
df.loc[(df.Level == 'Beginner')].BodyPart.unique()

['Abdominals', 'Abductors', 'Adductors', 'Biceps', 'Calves', ..., 'Lower Back', 'Neck', 'Quadriceps', 'Shoulders', 'Triceps']
Length: 15
Categories (17, object): ['Abdominals', 'Abductors', 'Adductors', 'Biceps', ..., 'Quadriceps', 'Shoulders', 'Traps', 'Triceps']

In [10]:
beginner = df.loc[(df.Level == 'Beginner')]

for i in beginner.BodyPart.unique():
    print(beginner.loc[(df.BodyPart == i)].BodyPart.unique()[0], beginner.loc[(df.BodyPart == i)].shape)

Abdominals (15, 5)
Abductors (3, 5)
Adductors (2, 5)
Biceps (1, 5)
Calves (1, 5)
Chest (20, 5)
Forearms (1, 5)
Glutes (6, 5)
Hamstrings (9, 5)
Lats (2, 5)
Lower Back (2, 5)
Neck (3, 5)
Quadriceps (14, 5)
Shoulders (1, 5)
Triceps (2, 5)
